<a href="https://colab.research.google.com/github/Anush1214/AML-PROJ/blob/main/Homechatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages if not already installed
!pip install gradio spacy speechrecognition pydub

import gradio as gr
import pandas as pd
import spacy
import speech_recognition as sr
from io import BytesIO
from pydub import AudioSegment

# Load spaCy model
import sys
try:
    nlp = spacy.load('en_core_web_sm')
except:
    import subprocess
    subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load('en_core_web_sm')

# Load the home remedies dataset
df = pd.read_csv('minor_diseases_home_remedies.csv')
df['disease'] = df['disease'].str.lower()
df['symptoms'] = df['symptoms'].str.lower()

def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

def match_disease(user_input):
    tokens = preprocess(user_input)
    # Try to match disease name
    for idx, row in df.iterrows():
        disease_tokens = preprocess(row['disease'])
        if any(token in tokens for token in disease_tokens):
            return row
    # Try to match symptoms
    for idx, row in df.iterrows():
        symptom_tokens = preprocess(row['symptoms'])
        if any(token in tokens for token in symptom_tokens):
            return row
    return None

def chatbot_response(message, history):
    result = match_disease(message)
    if result is not None:
        response = (
            f"**Possible condition:** {result['disease'].capitalize()}\n"
            f"**Common symptoms:** {result['symptoms']}\n"
            f"**Suggested home remedies:** {result['remedy']}"
        )
    else:
        response = "Sorry, I couldn't find a matching condition. Please try describing your symptoms differently or consult a doctor."
    history = history or []
    history.append((message, response))
    return history, history

def speech_to_text(audio):
    if audio is None:
        return ""
    # Convert the audio to wav for speech_recognition
    audio_bytes = BytesIO(audio)
    sound = AudioSegment.from_file(audio_bytes)
    wav_io = BytesIO()
    sound.export(wav_io, format="wav")
    wav_io.seek(0)
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_io) as source:
        audio_data = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio_data)
    except sr.UnknownValueError:
        text = ""
    except sr.RequestError:
        text = ""
    return text

with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 🩺 Home Remedies Healthcare Chatbot
        Describe your symptoms in text or use your voice. The chatbot will suggest possible minor conditions and home remedies.
        """
    )
    chatbot = gr.Chatbot(label="Chat", height=400)
    with gr.Row():
        txt = gr.Textbox(show_label=True, placeholder="Type your symptoms here...", scale=4)
        audio = gr.Audio(sources=["microphone"], type="filepath", label="Speak your symptoms", scale=1)
    clear = gr.Button("Clear Chat")

    def handle_text_submit(message, history):
        return chatbot_response(message, history)

    def handle_audio_submit(audio_file, history):
        if audio_file is None:
            return history, history
        text = speech_to_text(audio_file)
        if not text:
            response = "Sorry, I couldn't understand the audio. Please try again."
            history = history or []
            history.append(("Voice Input", response))
            return history, history
        return chatbot_response(text, history)

    txt.submit(handle_text_submit, [txt, chatbot], [chatbot, chatbot])
    audio.change(handle_audio_submit, [audio, chatbot], [chatbot, chatbot])
    clear.click(lambda: ([], []), None, [chatbot, chatbot])

demo.launch()


<ipython-input-5-eae52af3e4be>:85: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat", height=400)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d0fd2da86f2043657.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
